In [1]:
import io, os
import numpy as np
from google.cloud import vision
from PIL import ImageTk, Image, ImageDraw
import pandas as pd
import sqlite3
import json

In [2]:
# create client
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="static/cloud_credentials_steel-operative-337913-b3f7be97f534.json"
client = vision.ImageAnnotatorClient()

In [3]:
img_path = "static/images/paintings/"
paintings = os.listdir(img_path)

In [4]:
con = sqlite3.connect('rijksstudio.db')
cur = con.cursor()

In [5]:
objects_vision = []

In [6]:
def get_object_data(obj, painting, dims):
    normalized_vertices = obj.bounding_poly.normalized_vertices
    
    q = cur.execute("""INSERT OR REPLACE INTO Objects 
               VALUES (?,?,?,?,?,?,?,?,?,?);""", (painting, 
                                              obj.name.lower(), 
                                              obj.score, 
                                              dims[0] * normalized_vertices[0].x, 
                                              dims[1] * normalized_vertices[0].y, 
                                              dims[0] * normalized_vertices[2].x, 
                                              dims[1] * normalized_vertices[2].y, 
                                              "GOOGLE_VISION",
                                              0,
                                              0))
    
    con.commit()
    
    return {
      'painting_id': painting,
      'object_name': obj.name.lower(),
      'confidence': obj.score,
      'min_x': dims[0] * normalized_vertices[0].x,
      'min_y': dims[1] * normalized_vertices[0].y,
      'max_x': dims[0] * normalized_vertices[2].x,
      'max_y': dims[1] * normalized_vertices[2].y
    }

In [7]:
# create objects table
cur.execute("""CREATE TABLE IF NOT EXISTS Objects
               (painting_id TEXT,
                object_name TEXT,
                confidence INTEGER,
                min_x INTEGER, 
                min_y INTEGER,
                max_x INTEGER, 
                max_y INTEGER,
                source TEXT,
                total_clicks INTEGER DEFAULT 0,
                flags INTEGER DEFAULT 0
                )""")

# Save changes
con.commit()

In [8]:
for painting in paintings:
    print("Reading "+painting+" ...")
    # Loads the image into memory
    with io.open(os.path.join(img_path, painting), 'rb') as image_file:
        content = image_file.read()
        dims = Image.open(image_file).size
    
    # call API
    image = vision.Image(content=content)
    response = client.object_localization(image=image)
    localized_object_annotations = response.localized_object_annotations
    
    # add every object data to db
    for obj in localized_object_annotations:
        # only keep object with confidence of 0.9 or higher
        if obj.score<0.9:
            continue
        normalized_vertices = obj.bounding_poly.normalized_vertices
        
        objects_vision.append(get_object_data(obj, painting, dims))

Reading RP-F-2004-71.png ...
Reading RP-F-2014-7-1-3.png ...
Reading RP-P-H-1086.png ...
Reading RP-T-1881-A-107.png ...
Reading RP-T-1951-200.png ...
Reading RP-T-1951-203.png ...
Reading SK-A-1107.png ...
Reading SK-A-113.png ...
Reading SK-A-1130.png ...
Reading SK-A-129.png ...
Reading SK-A-137.png ...
Reading SK-A-1451.png ...
Reading SK-A-1595.png ...
Reading SK-A-1627.png ...
Reading SK-A-1649.png ...
Reading SK-A-1751.png ...
Reading SK-A-180.png ...
Reading SK-A-1857.png ...
Reading SK-A-199.png ...
Reading SK-A-2150.png ...
Reading SK-A-2152.png ...
Reading SK-A-2180.png ...
Reading SK-A-2205.png ...
Reading SK-A-2213.png ...
Reading SK-A-2251.png ...
Reading SK-A-2344.png ...
Reading SK-A-2362.png ...
Reading SK-A-2366.png ...
Reading SK-A-2388.png ...
Reading SK-A-2452.png ...
Reading SK-A-2549.png ...
Reading SK-A-2565.png ...
Reading SK-A-263.png ...
Reading SK-A-2713.png ...
Reading SK-A-2836.png ...
Reading SK-A-2962.png ...
Reading SK-A-3059.png ...
Reading SK-A-3103.p

In [9]:
con.close()

In [11]:
# Save objects into json
json.dump(objects_vision, open("objects_vision.json", 'w'))